# Punctual Python - A Potpourri of Time-Sensitive Materials

Daylight Saving Time is over... *for now*. But that doesn't mean we get to stop thinking about time!

The various methods humans have invented to measure, represent, and delineate time may appear simple at first pass, but seem to get more and more complex and unintuitive the longer you look at them.

Why do some states use Daylight Saving Time, but not others? Why do European countries use Day-Month-Year formatting for most dates, while the U.S. uses Month-Day-Year? Why does China only have one time zone when it's over 3000 miles wide between its Eastern and Western borders? What's up with Australia having time zones that are offset by 30-minute increments instead of just whole hours?

![Australia (Standard Time)](media/australia_standard.png) ![Australia (Daylight Saving Time)](media/australia_daylight.png)

Unfortunately, I won't be answering many "Why" questions about time policies in international governments today, but I will provide you with some Python tools you can use for more procedural problems, like:

* switching between different time formats when cleaning data
* converting UTC timestamps to local time
* timing how long code takes to execute
* estimating the time left for code execution when processing items in an iterable object
* inserting pauses between steps of a process.
* playing sampled audio at intervals


## Imports

In [72]:
#Python Standard Library modules
import calendar
from datetime import datetime
import time
import multiprocessing
import zoneinfo

#External modules
import dateutil
from tqdm import tqdm
import pytz
import tzdata
import pygame

import pandas as pd
import numpy as np

/opt/anaconda3/envs/workshops/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.32.56, Python 3.13.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Time in Computer Systems

### Unix Epoch Time

For us humans, it makes sense to think of time in terms of units with subdivisions... 60 seconds to a minute, 60 minutes to an hour, 24 hours to a day, 525600 minutes to a year, etc.

However, it's much easier for *computers* to measure time if seconds are the only unit. During the development of the AT&T Unix operating system at Bell Labs in 1969, researchers decided to start counting time from midnight of January 1st, 1970. The resulting "Unix time" or "Epoch time" is usually represented as an integer, but sometimes includes a decimal component as well to capture microseconds or milliseconds.

As I wrote this block of text on Monday, October 6th, 2025, at 10:42 in the morning, Pacific Daylight Time, the Unix Epoch time was: 1759772530. It's been *~1.76 billion seconds* since January 1, 1970. We won't hit the 2-billion mark until 2033.

### `time.time()`

The `time` module in the Python Standard Library has a function called `time` that can get the current time and return it as a Unix timestamp, with microseconds and milliseconds included.

In [78]:
time.time()

1761849157.98159

By itself, this doesn't seem to do much, but if we store the value in a variable, we can compare the stored value to another instance of `time.time()` later to see how much time has elapsed between them. 

#### Exercise - Difference Between Two Times:

Try running the cells below, pausing for a moment between them.

In [82]:
t = time.time()
print(t)

1761849279.009364


In [83]:
s = time.time()
print(s)
print(s - t)

1761849281.0250938
2.0157299041748047


### ISO-8601

Another solution for handling representation of time is the ISO-8601 format. ("ISO" is the International Organization for Standardization). Unlike Unix Epoch time, this format is meant for humans to read, but it's specifically intended to eliminate ambiguities when communicating and exchanging data. 

The time increments in ISO read from largest to smallest, meaning that sorting ISO-8601 dates alphabetically (or, to be more precise, lexicographically) is the same as sorting them chronologically.

The date is `YYYY-MM-DD`, then there is a `T` that indicates the division between date and the time, which is written `HH:MM:SS` followed by a `.` and then 3 digits for milliseconds, followed by a `Z`.

Monday, October 6th, 2025, at 10:42 in the morning, Pacific Daylight Time, usually reads something like this:

`2025-10-06T17:42:00.000Z`

Hold on... something is odd about that... That's seven hours ahead of 10:42...

### UTC

ISO-8601 is anchored on Coordinated Universal Time (UTC), a successor to Greenwich Mean Time (The time zone of the Royal Observatory in Greenwich, London). Pacific Daylight Time is equivalent to UTC minus 7 hours, or a "UTC Offset" of `UTC-7:00`.

*However*, now that Daylight Saving Time is over in California, we are back to Pacific Standard Time, with a UTC Offset of `UTC-8:00`.

A full list of UTC offsets may be found [here](https://en.wikipedia.org/wiki/List_of_UTC_offsets).

Our neighbor state Arizona does not observe Daylight Saving Time, so back in October, when I began to assemble this workshop, California and Arizona each had `UTC-7:00` as their UTC Offset. Since CA and AZ are now both using Standard Time, we are now a time zone apart.

This can create logistical hurdles for companies that operate in both California and Arizona, but it's still not as big a problem as what Australia has to deal with across its territories.

## Handling Date and Time with `time`, `datetime` and `calendar`

The Python Standard Library has several modules that deal with dates and times. `time`, `datetime`, and `calendar` are imperfect, but still handy enough to work for most purposes. 

For those times when these modules are not sufficient, we can use the external module `dateutil`, which fills in many gaps in functionality.

### `strftime` and `strptime`

In [18]:
type(t)

float

In [19]:
type(time.localtime(t))

time.struct_time

In [20]:
time.strftime('%Y-%m-%d %H:%M %Z', time.localtime(t))

'2025-10-29 14:41 PDT'

The `time` struct_time and `datetime` datetime modules store time units in slightly different ways:

In [21]:
time.strptime('2025-10-06 11:07 PDT', '%Y-%m-%d %H:%M %Z')

time.struct_time(tm_year=2025, tm_mon=10, tm_mday=6, tm_hour=11, tm_min=7, tm_sec=0, tm_wday=0, tm_yday=279, tm_isdst=1)

In [22]:
datetime.today()

datetime.datetime(2025, 10, 29, 15, 46, 52, 768677)

`datetime.today()` is a simple way of getting a snapshot of the current time. We can do this twice to see how long something takes.

In [23]:
a = datetime.today()

time.sleep(1.5)

b = datetime.today()

b - a

datetime.timedelta(seconds=1, microseconds=505491)

We might expect the difference between b and a to be exactly 1.5 seconds, but it takes some amount of time for the functions to run. This number will be different between different executions of the code.

## `timeit`

There's another way of timing code execution time, though: `timeit`.

In [24]:
timeit.timeit("time.sleep(1.5)", number=1)

1.5045748339998681

## Time Zones

In [88]:
datetime.today().astimezone().tzinfo

datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'PDT')

In [90]:
unix_epoch_time = 1759772530

t = datetime.fromtimestamp(unix_epoch_time).astimezone()

print(t)

datetime.strftime(t, '%Y-%m-%d')

2025-10-06 10:42:10-07:00


'2025-10-06'

In [44]:
a = pytz.timezone("Asia/Kolkata")
b = datetime.now(a)
print(b)

2025-10-30 04:26:11.631259+05:30


In [44]:
a = pytz.timezone("Asia/Kolkata")
b = datetime.now(a)
print(b)

2025-10-30 04:26:11.631259+05:30


## `zoneinfo` - Python Standard Library Time Zone Module

## `pytz` - A Third-Party Time Zone Module

In [62]:
zonelist1 = []

for tz in pytz.all_timezones:
    zonelist1.append(tz)

## `tzdata` - A Newer Third-Party Time Zone Module

In [64]:
zonelist2 = []
for tz in zoneinfo.available_timezones():
    zonelist2.append(tz)

In [65]:
zonelist1 == zonelist2

False

In [66]:
len(zonelist1)

597

In [67]:
len(zonelist2)

599

## Parsing Dates the Easy Way with `dateutil.parser`

After using `datetime`, `dateutil` is a relative breeze. Formatting dates becomes incredibly easy with `dateutil`'s `parser`.

Please note that the default behavior is to assume Month-Day-Year formatting unless the optional "dayfirst" argument is set to `True`.

In [12]:
days = [
    'September 13th 2009',
    '9 2 2024',
    '2-9-2024',
    '31 5 2024',
    '12th August 2018'
]

for day in days:
    day = dateutil.parser.parse(day)
    print(day)
    print(day.strftime("%Y-%m-%d"))

2009-09-13 00:00:00
2009-09-13
2024-09-02 00:00:00
2024-09-02
2024-02-09 00:00:00
2024-02-09
2024-05-31 00:00:00
2024-05-31
2018-08-12 00:00:00
2018-08-12


In [13]:
days[0]

'September 13th 2009'

In [14]:
day.strftime("%Y-%m-%d")

'2018-08-12'

## Estimating Time Remaining with `tqdm`

In [15]:
for x in tqdm(range(60)):
    
    time.sleep(0.1)
    

100%|███████████████████████████████████████████| 60/60 [00:06<00:00,  9.51it/s]


In [16]:
time_list

for x in tqdm(range(60)):
    
    time.sleep(0.1)
    

NameError: name 'time_list' is not defined

# MTA Timestamps

In [71]:
staten_island = pd.read_csv('staten_island.csv', index_col=0)
staten_island

NameError: name 'pd' is not defined

# Time in Music

## The `pygame` Module

Python is by *far* not the best programming language for game design, but the `pygame` module still has some useful features for creating user interfaces, including an audio mixer.

In [2]:
## Initialize the mixer:

pygame.mixer.init()

## The Winstons - "Amen, Brother"

In the "Amen" folder, there are a selection of `.wav` files, all individual drum hits from the first measure of a famous drum solo by Gregory C. Coleman of [The Winstons](https://en.wikipedia.org/wiki/The_Winstons), on a 1969 track called "Amen, Brother".

This short, mid-song drum solo, (often called a "break" or "breakbeat" since it takes place during a break in other instrumentation) is one of the most-sampled music recordings of all time. Whether or not you're aware of it, you've probably heard it before. It has spawned entire genres of electronic music. 

Neither Gregory Coleman nor any of the other Winstons members ever received royalties for the drum loop's use in *thousands* of published works of music.

* ***If you care to dive down a rabbit hole about music culture, sampling, intellectual property, and the public domain, [Nate Harrison's video essay "Can I Get an Amen?" from 2004](https://www.youtube.com/watch?v=XPoxZW8JzzM) is worth a watch.***

## Drum Samples

In [3]:
kick1 = 'Amen/Amen 1-1 (Kick).wav'
kick2 = 'Amen/Amen 1-2 (Kick).wav'
snare1 = 'Amen/Amen 1-3 (Snare).wav'
hat1 = 'Amen/Amen 1-4a (Hat).wav'
snarelite1 =  'Amen/Amen 1-4b (Snarelite).wav'
hat2 = 'Amen/Amen 1-5a (Hat).wav'
snarelite2 = 'Amen/Amen 1-5b (Snarelite).wav'
kickride = 'Amen/Amen 1-6a (Kickride).wav'
kick3 = 'Amen/Amen 1-6b (Kick).wav'
snare2 = 'Amen/Amen 1-7 (Snare).wav'
hat3 = 'Amen/Amen 1-8a (Hat).wav'
snarelite3 = 'Amen/Amen 1-8b (Snarelite).wav'

In [4]:
pygame.mixer.music.load(snare1)
pygame.mixer.music.play()

### Note Values

If you've never dealt with music theory before, don't worry about this too much, but most music is divided into short segments called "measures" that contain a specified number of beats (regularly-spaced rhythmic pulses). A "4/4" time signature or "Common time" as it is sometimes called represents four beats to a measure, each with a value of a "quarter note". The rhythm is felt in groups of four.

Tempo is a measure of how many beats there are in a given time period. In the original sample from "Amen, Brother", the tempo is a little under 140 beats per minute. We're going to use 140 as a starting point.

In [5]:
# Define beats per minute
bpm = 140

# Define note lengths
half = 30 / bpm
quarter = 60 / bpm
eighth = 60 / bpm / 2
sixteenth = 60 / bpm / 4
thirtysecond = 60 / bpm / 8
sixtyfourth = 60 / bpm / 16

In [6]:
eighth

0.21428571428571427

In [7]:
sixteenth

0.10714285714285714

An eighth note (half of a quarter note, or half of a "beat" in 4/4 time) played at a tempo of 140 beats per minute is ~0.21429 seconds long. A sixteenth note is ~0.10714 seconds. If we know these note values, we can send them to `time.sleep()` to instruct Python to wait after playing a sample for that duration before playing the next sample.

This lets us play the audio samples at a consistent tempo, even accounting for different note values.

In [8]:
def play_sample(sample, duration=eighth):
    pygame.mixer.music.load(sample)
    pygame.mixer.music.play()
    print(sample)
    time.sleep(duration)

In [9]:
def play_measure(samples, durations):
    for sample, duration in zip(samples, durations):
        play_sample(sample, duration)

In [10]:
# 1 measure
# 4 eighth notes and 8 sixteenth notes comprise a full measure... just add up the fractions!
# The snare drums fall on the 2nd and 4th quarter notes of the measure

samples = [
    kick1,
    kick2, 
    snare1, 
    hat1, snarelite1, 
    hat2, snarelite2, 
    kickride, kick3, 
    snare2, 
    hat3, snarelite3
]

durations = [
    eighth, 
    eighth, 
    eighth, 
    sixteenth, sixteenth, 
    sixteenth, sixteenth,
    sixteenth, sixteenth,
    eighth, 
    sixteenth, sixteenth]

In [11]:
for i in range(4):
    play_measure(samples, durations)

Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-5b (Snarelite).wav
Amen/Amen 1-6a (Kickride).wav
Amen/Amen 1-6b (Kick).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-8a (Hat).wav
Amen/Amen 1-8b (Snarelite).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-5b (Snarelite).wav
Amen/Amen 1-6a (Kickride).wav
Amen/Amen 1-6b (Kick).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-8a (Hat).wav
Amen/Amen 1-8b (Snarelite).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-5b (Snarelite).wav
Amen/Amen 1-6a (Kickride).wav
Amen/Amen 1-6b (Kick).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-8a (Hat).wav
Amen/Amen 1-8b (Snarelite).wav
Amen/Amen 1-1 (Kick).w

### "Swing Time"

To change the rhythmic feel, or "groove", you can mimic swing time or swung rhythm by changing the times of sixteenth notes depending on where they fall in the measure. A sixteenth note that lines up with eighth-note subdivisions will be slightly longer in duration, but a sixteenth note that falls off of an eighth-note subdivision will have a shorter duration.

In [12]:
swing = 0.15

sixteenth * (1+swing)+ sixteenth * (1-swing)

0.21428571428571425

In [13]:
eighth

0.21428571428571427

In [14]:
# 1 measure

swing = 0.15

samples = [
    kick1,
    kick2, 
    snare1, 
    hat1, snarelite1, 
    hat2, snarelite2, 
    kickride, kick3, 
    snare2, 
    hat3, snarelite3
]

durations = [
    eighth, 
    eighth, 
    eighth, 
    sixteenth * (1+swing), sixteenth * (1-swing), 
    sixteenth * (1+swing), sixteenth * (1-swing),
    sixteenth * (1+swing), sixteenth * (1-swing),
    eighth, 
    sixteenth * (1+swing), sixteenth * (1-swing)]

In [15]:
for i in range(4):
    play_measure(samples, durations)

Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-5b (Snarelite).wav
Amen/Amen 1-6a (Kickride).wav
Amen/Amen 1-6b (Kick).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-8a (Hat).wav
Amen/Amen 1-8b (Snarelite).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-5b (Snarelite).wav
Amen/Amen 1-6a (Kickride).wav
Amen/Amen 1-6b (Kick).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-8a (Hat).wav
Amen/Amen 1-8b (Snarelite).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-5b (Snarelite).wav
Amen/Amen 1-6a (Kickride).wav
Amen/Amen 1-6b (Kick).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-8a (Hat).wav
Amen/Amen 1-8b (Snarelite).wav
Amen/Amen 1-1 (Kick).w

## Jungle/Drum'n'Bass

The Winstons' drum sample became the basis for several genres of electronic music by virtue of its rhythmic feel and distinct timbre. In the early 1990s, the sample was used extensivley by music producers in the Carribean and the U.K. in fast-paced dance music that chopped up the beat and played with its syncopation.

In [16]:
# Define beats per minute
bpm = 176

# Define note lengths
half = 30 / bpm
quarter = 60 / bpm
eighth = 60 / bpm / 2
sixteenth = 60 / bpm / 4
thirtysecond = 60 / bpm / 8
sixtyfourth = 60 / bpm / 16

In [17]:
# Measures 1 & 3

samples1 = [
    kick1,
    hat1, hat2,
    snare1, 
    kick1,
    hat1, snarelite1,
    snare1,
    kick1, 
    kick2
]

durations1 = [
    eighth, 
    sixteenth, sixteenth,
    eighth, 
    eighth, 
    sixteenth, sixteenth,
    eighth, 
    eighth, 
    eighth]

In [18]:
# Measure 2

samples2 = [
    kick1,
    hat1, hat2,
    snare1, 
    kick1,
    hat1, snarelite1,
    snare1,
    kick1, snarelite1,
    snare2
]

durations2 = [
    eighth, 
    sixteenth, sixteenth,
    eighth, 
    eighth, 
    sixteenth, sixteenth,
    eighth, 
    sixteenth, sixteenth, 
    eighth]

In [19]:
# Measure 4

samples3 = [
    kick1,
    hat1, hat2,
    snare1, 
    hat1, snarelite1,
    hat2, snarelite2,
    kick1,
    snare1, snare2,
    snare1, snare2
]

durations3 = [
    eighth, 
    sixteenth, sixteenth,
    eighth, 
    sixteenth, sixteenth, 
    sixteenth, sixteenth,
    eighth, 
    sixteenth, sixteenth, 
    sixteenth, sixteenth]

In [20]:
play_measure(samples1, durations1)
play_measure(samples2, durations2)
play_measure(samples1, durations1)
play_measure(samples3, durations3)
play_sample(kick1, eighth)

Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-7 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-2 (Kick).wav
Amen/Amen 1-1 (Kick).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-3 (Snare).wav
Amen/Amen 1-4a (Hat).wav
Amen/Amen 1-4b (Snarelite).wav
Amen/Amen 1-5a (Hat).wav
Amen/Amen 1-

Using dedicated Digital Audio Workstation software like GarageBand, FL Studio, Reason, Cubase, Live, Logic, or even ProTools is likely an easier route to producing music, but if you really like building things from scratch, Python can work, too!